# 使用「回溯法」(backtracking)的框架
# 解    數    獨！

In [1]:
# 數獨：一般來說是9*9的格子中
# 每格填入1~9其中一數
# 但是須符合以下規則：
# 對於每一行、每一列、以及每一個3*3小方格
# 1~9都必須且只能出現一次！

In [2]:
# 以下的版本在不同大小的數獨都行的通
# 前提是邊長格數平方根需為整數
# 例如4*4, 9*9, 16*16, ...

### 以下為回溯法的框架：

In [5]:
def do_backtrack(a:list, inputs:list):
    c = []
    if (is_a_solution(a, inputs)):
        process_solution(a, inputs)
    else:
        construct_candidate(a, inputs, c)
        for i in c:
            a.append(i)
            do_backtrack(a, inputs)
            a.pop()

### 判斷目前的解向量"a"是否為可能解：用長度判斷
### 注意到這裡的解向量a其中的元素
### 是要填入「空格」的數字，按列依序填入
### 因此我們先計算出空格的數量，若a的長度達到空格數就為正解
### (空格在題目中以數字0填充表示)

In [6]:
def is_a_solution(a, inputs):
    n = len(inputs)
    count = 0
    for i in range(n):
        for j in range(n):
            if inputs[i][j] == 0:
                count += 1
    return len(a) == count

## 以下為核心的子程式！
### 用來產生下一個
### 添加到解向量中的候選解的程式
### 所產生的候選必須考慮目前解向量的答案
### 依照問題敘述中的規則

In [8]:
"""
想法與作法詳述：
首先將題目與目前的解向量各複製一份
並對副本操作，防止被汙染到。
因此以下題目與解向量操作都是作用在副本喔！

首先將第一個迴圈跑過題目的所有格子
只要發現有空格(以0表示)，就填入目前解向量的答案
一格一格進去，直到把解向量a填完
此時我們就得到一個填入a的部分解了

第二個迴圈是求出這個部分解的第一個空格在哪
也就是說，求出我們下一個需要填的空格是哪個

最後這個大迴圈就是重頭戲啦！
試著把每個數字填入並判斷，不合以下條件(有重複數字)的剔掉：
1.目前部分解中，這一列已經有這個數字了(第一個if)
2.目前部分解中，這一欄已經有這個數字了(第二個if)
3.目前部分解中，這一3*3的小九宮格已經有這個數字了(第三個if)
第三個if之前的部分，是在求出目前要填的空格屬於哪個小九宮格
並且把這個九宮格的左上角位置抓出來
然後就可以輕鬆把九宮格其他部分也跟著抓出來了
"""

'\n想法與作法詳述：\n首先將題目與目前的解向量各複製一份\n並對副本操作，防止被汙染到。\n因此以下題目與解向量操作都是作用在副本喔！\n\n首先將第一個迴圈跑過題目的所有格子\n只要發現有空格(以0表示)，就填入目前解向量的答案\n一格一格進去，直到把解向量a填完\n此時我們就得到一個填入a的部分解了\n\n第二個迴圈是求出這個部分解的第一個空格在哪\n也就是說，求出我們下一個需要填的空格是哪個\n\n最後這個大迴圈就是重頭戲啦！\n試著把每個數字填入並判斷，不合以下條件(有重複數字)的剔掉：\n1.目前部分解中，這一列已經有這個數字了(第一個if)\n2.目前部分解中，這一欄已經有這個數字了(第二個if)\n3.目前部分解中，這一3*3的小九宮格已經有這個數字了(第三個if)\n第三個if之前的部分，是在求出目前要填的空格屬於哪個小九宮格\n並且把這個九宮格的左上角位置抓出來\n然後就可以輕鬆把九宮格其他部分也跟著抓出來了\n'

In [7]:
def construct_candidate(a, inputs, c):
    partial = [x[:] for x in inputs.copy()]
    b = a.copy()
    n, k = len(inputs), len(a)
    q = int(n ** 0.5)
    for i in range(n ** 2):
        ridx = i // n
        cidx = i % n
        if partial[ridx][cidx] == 0 and len(b) > 0:
            partial[ridx][cidx] = b.pop(0)
    for i in range(n ** 2):
        ridx = i // n
        cidx = i % n
        if partial[ridx][cidx] == 0:
            break
    for t in range(n):
        i = t + 1
        legal = True
        if i in partial[ridx]:
            legal = False
        if not all([str(partial[j][cidx]) if partial[j][cidx] != i else '' for j in range(n)]):
            legal = False
        ul_row = (ridx // q) * q
        ul_col = (cidx // q) * q
        l = []
        for m in range(q):
            l += partial[ul_row][ul_col:ul_col + q]
            ul_row += 1
        if i in l:
            legal = False
        if legal:
            c.append(i)

### 若該解向量a為正解，先複製一份題目
### 把正解a按列依序填入空格

In [9]:
def process_solution(a:list, inputs:list):
    global solution
    solution = [x[:] for x in inputs.copy()]
    index, n = 0, len(inputs)
    for i in range(n):
        for j in range(n):
            if solution[i][j] == 0 and index < len(a):
                solution[i][j] = a[index]
                index += 1

### 以下單純用做檢查這個解答是否正確無誤

In [10]:
import math
def is_valid_sudoku(partial_assignment:list)->bool:
    #Return True if subarray partial_assignment[start_row:end_row][start_col:end_coL]
    #contains any duplicates in {1, 2, ..., len(partial_assignnent)};
    #otherwise return False.
    def has_duplicate(block):
        block = list(filter(lambda x: x!=0, block))
        return len(block) != len(set(block)) 
    n = len(partial_assignment)
    #Check row and column constraints.
    if any(has_duplicate([partial_assignment[i][j] for j in range(n)])
           or has_duplicate([partial_assignment[j][i] for j in range(n)])
           for i in range(n)):
        return False
    #Check region constraints.
    region_size = int(math.sqrt(n))
    return all(not has_duplicate([
        partial_assignment[a][b]
        for a in range(region_size * I, region_size * (I+1))
        for b in range(region_size * J, region_size * (J+1))])
        for I in range(region_size) for J in range(region_size))

def check_solution(sol:list, init:list)->str:
    n = len(sol)
    #Check empty slots
    for r in range(n):
        for c in range(n):
            if sol[r][c] == 0:
                return "There has empty slot(s) in solution."
    #Check valid or not
    if (sol == [] or not is_valid_sudoku(sol)):
        return "It's not a valid solution!"
    #Check solving from init grid
    for r in range(n):
        for c in range(n):
            if init[r][c] != 0 and sol[r][c] != init[r][c]:
                return "It's not solved from initial grid."
    return "pass!"

# 實測結果

In [11]:
solution = []
sudoku = [[0, 3, 2, 0, 0, 0, 8, 0, 4], [8, 0, 0, 2, 0, 0, 0, 7, 0], [0, 1, 7, 0, 0, 5, 9, 0, 6], [5, 8, 0, 0, 2, 0, 0, 3, 0], [0, 0, 6, 0, 4, 0, 7, 0, 0], [0, 0, 4, 9, 1, 3, 0, 6, 0], [0, 0, 0, 7, 3, 0, 2, 0, 0], [0, 5, 9, 0, 0, 0, 0, 0, 1], [1, 0, 0, 8, 0, 9, 0, 0, 0]]
do_backtrack([],sudoku)
print(check_solution(solution, sudoku))
print(solution)

solution = []
sudoku = [[0, 5, 7, 0, 2, 3, 0, 1, 6], [0, 0, 0, 7, 0, 9, 5, 0, 0], [0, 0, 2, 0, 0, 0, 0, 8, 0], [0, 0, 0, 0, 0, 0, 0, 3, 0], [3, 0, 0, 8, 0, 6, 4, 0, 9], [2, 0, 9, 0, 0, 0, 0, 0, 7], [0, 2, 0, 9, 0, 1, 0, 0, 5], [9, 0, 0, 3, 6, 0, 8, 0, 1], [0, 0, 6, 0, 7, 0, 0, 0, 4]]
do_backtrack([],sudoku)
print(check_solution(solution, sudoku))
print(solution)

solution = []
sudoku = [[8, 0, 0, 3, 2, 0, 0, 7, 0], [0, 5, 7, 8, 0, 0, 9, 0, 0], [0, 9, 0, 0, 0, 1, 0, 4, 0], [0, 7, 8, 0, 1, 0, 6, 0, 3], [2, 0, 0, 0, 7, 0, 1, 0, 0], [9, 0, 4, 5, 0, 3, 0, 0, 0], [0, 0, 5, 6, 0, 0, 0, 0, 9], [0, 6, 0, 0, 8, 0, 7, 0, 1], [0, 0, 3, 0, 0, 4, 8, 0, 0]]
do_backtrack([],sudoku)
print(check_solution(solution, sudoku))
print(solution)

pass!
[[9, 3, 2, 1, 7, 6, 8, 5, 4], [8, 6, 5, 2, 9, 4, 1, 7, 3], [4, 1, 7, 3, 8, 5, 9, 2, 6], [5, 8, 1, 6, 2, 7, 4, 3, 9], [3, 9, 6, 5, 4, 8, 7, 1, 2], [2, 7, 4, 9, 1, 3, 5, 6, 8], [6, 4, 8, 7, 3, 1, 2, 9, 5], [7, 5, 9, 4, 6, 2, 3, 8, 1], [1, 2, 3, 8, 5, 9, 6, 4, 7]]
pass!
[[8, 5, 7, 4, 2, 3, 9, 1, 6], [6, 1, 3, 7, 8, 9, 5, 4, 2], [4, 9, 2, 6, 1, 5, 7, 8, 3], [5, 6, 4, 2, 9, 7, 1, 3, 8], [3, 7, 1, 8, 5, 6, 4, 2, 9], [2, 8, 9, 1, 3, 4, 6, 5, 7], [7, 2, 8, 9, 4, 1, 3, 6, 5], [9, 4, 5, 3, 6, 2, 8, 7, 1], [1, 3, 6, 5, 7, 8, 2, 9, 4]]
pass!
[[8, 4, 1, 3, 2, 9, 5, 7, 6], [3, 5, 7, 8, 4, 6, 9, 1, 2], [6, 9, 2, 7, 5, 1, 3, 4, 8], [5, 7, 8, 4, 1, 2, 6, 9, 3], [2, 3, 6, 9, 7, 8, 1, 5, 4], [9, 1, 4, 5, 6, 3, 2, 8, 7], [1, 8, 5, 6, 3, 7, 4, 2, 9], [4, 6, 9, 2, 8, 5, 7, 3, 1], [7, 2, 3, 1, 9, 4, 8, 6, 5]]
